#### Task 1

In [28]:
from dataclasses import dataclass

@dataclass
class Plot:
    value: str
    seen: bool = False

In [30]:
field = []
with open("input.txt", "r") as in_file:
    for line in in_file.readlines():
        line = line.strip()
        if len(line) >= 1:
            field.append(list(map(lambda x: Plot(x), line)))

In [31]:
def in_range(pos, field):
    if pos[0] < 0 or pos[0] >= len(field[0]):
        return False
    if pos[1] < 0 or pos[1] >= len(field):
        return False
    return True

def get_value(pos, field):
    return field[pos[1]][pos[0]].value

def all_neighbours(pos):
    return [(x+pos[0], pos[1]) for x in [-1, 1]] + [(pos[0], y+pos[1]) for y in [-1, 1]]

def valid_neighbours(pos, field):
    candidates = [(x+pos[0], pos[1]) for x in [-1, 1]] + [(pos[0], y+pos[1]) for y in [-1, 1]]
    return [x for x in candidates if in_range(x, field)]

def get_region(start_pos, field):
    queue = [start_pos]
    elements = []
    while len(queue) > 0:
        next = queue.pop()
        elements.append(next)
        for neighbour in valid_neighbours(next, field):
            if get_value(neighbour, field) == get_value(start_pos, field) and not neighbour in queue and not neighbour in elements:
                queue.append(neighbour)

    return elements

def get_region_value(region: list[tuple[int, int]]):
    area = len(region)
    perimeter = 0
    for pos in region:
        for neighbour in all_neighbours(pos):
            if not neighbour in region:
                perimeter += 1
    return area * perimeter

In [32]:
all_regions = []
for x in range(len(field[0])):
    for y in range(len(field)):
        if not field[y][x].seen:
            reg = get_region((x, y), field)
            all_regions.append(reg)
            for pos in reg:
                field[pos[1]][pos[0]].seen = True

total = 0
for reg in all_regions:
    total += get_region_value(reg)

print(total)

1930


#### Task 2

In [36]:
def build_group(groups: list[tuple[int, int]]):
    previous_len = 0
    while previous_len != len(groups):
        previous_len = len(groups)
        next_groups = []
        while len(groups) > 0:
            is_neighbouring = False
            for i in range(1, len(groups)):
                for ele in groups[0]:
                    for neighbour in all_neighbours(ele):
                        if neighbour in groups[i]:
                            is_neighbouring = True
                            next_groups.append(groups[0] + groups[i])
                            groups.pop(i)
                            groups.pop(0)
                            break
                    if is_neighbouring:
                        break
                if is_neighbouring:
                    break

            if not is_neighbouring:
                next_groups.append(groups.pop(0))


        groups = next_groups

    return groups

def get_side_amount(region: list[tuple[int, int]]) -> int:
    min_x = min(x[0] for x in region)
    max_x = max(x[0] for x in region)
    min_y = min(x[1] for x in region)
    max_y = max(x[1] for x in region)
    side_amount = 0
    for y in range(min_y, max_y+1):
        in_this_row = [x for x in region if x[1] == y]
        has_top_border = [[x] for x in in_this_row if not (x[0], x[1] - 1) in region]
        has_bot_border = [[x] for x in in_this_row if not (x[0], x[1] + 1) in region]

        side_amount += len(build_group(has_top_border)) + len(build_group(has_bot_border))

    for x in range(min_x, max_x + 1):
        in_this_col = [i for i in region if i[0] == x]
        has_left_border = [[i] for i in in_this_col if not (i[0] -1, i[1]) in region]
        has_right_border = [[i] for i in in_this_col if not (i[0] +1, i[1]) in region]

        side_amount += len(build_group(has_left_border)) + len(build_group(has_right_border))

    return side_amount

def get_region_value_sides(region):
    return len(region) * get_side_amount(region)

In [37]:
total = 0
for reg in all_regions:
    total += get_region_value_sides(reg)

print(total)

1206
